# Pretrained GLOVE

##  Load data

In [ ]:
x_poss = list(open("twitter-datasets/cleaned_data/cleaned_train_pos.txt", "r", encoding='utf-8').readlines())
x_poss = [s.strip() for s in x_poss]
x_pos = []
for elem in x_poss:
    if elem!='':
        tweet=''
        for word in elem.split(','):
            tweet+=word+' '
        x_pos.append(tweet)
x_negg = list(open("twitter-datasets/cleaned_data/cleaned_train_neg.txt", "r", encoding='utf-8').readlines())
x_negg = [s.strip() for s in x_negg]
x_neg = []
for elem in x_negg:
    if elem!='':
        tweet=''
        for word in elem.split(','):
            tweet+=word+' '
        x_neg.append(tweet)

In [ ]:
tesst = list(open("twitter-datasets/cleaned_data/cleaned_test_data.txt", "r", encoding='utf-8').readlines())
tesst = [s.strip() for s in tesst]
test = []
for elem in tesst:
    if elem!='':
        tweet=''
        for word in elem.split(','):
            tweet+=word+' '
        test.append(tweet)

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import GlobalAveragePooling1D
from keras.layers import Convolution1D
from keras.layers import MaxPooling1D
from keras.layers import Embedding
from keras.layers import LSTM
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np


In [ ]:
X = x_pos + x_neg
y = np.ones(len(x_pos)) + np.zeros(len(x_neg))

In [ ]:
maxs = []
for elem in X:
    maxs.append(len(elem.split(" ")))
max_ = max(maxs)

In [ ]:
MAX_LENGTH = 30
#MAX_LENGTH = max_
#MAX_LENGTH = 80

In [ ]:
def load_glove_embeddings(file):
    '''
    This method returns a dict with the word followed by it's vector
    '''
    word_embedding = {}
    f = open(file,'rb')
    for line in f:
        values = line.split()
        word_embedding[values[0]] = np.array([float(x) for x in values[1:]])
    f.close()
    return word_embedding

In [ ]:
word_embedding = load_glove_embeddings("glove.6B/glove.6B.200d.txt")

In [ ]:
tokenizer = Tokenizer(filters='')

In [ ]:
def create_sequences(max_, X, X_test):
    # fit train words to tokenizer
    tokenizer.fit_on_texts(X)
    # get words and their indexes
    word_index = tokenizer.word_index
    # number of words
    nb_word = len(word_index)
    print(nb_word)
    # transform text to sequence
    X_sequences = tokenizer.texts_to_sequences(X)
    test_sequences = tokenizer.texts_to_sequences(X_test)
    
    # Pad sequences
    X_sequences = sequence.pad_sequences(X_sequences, maxlen=MAX_LENGTH,padding='pre')
    test_sequences = sequence.pad_sequences(test_sequences, maxlen=MAX_LENGTH,padding='pre')

    # create y, first pos then neg
    train_size = len(X)
    y = np.array(int(train_size/2) * [1] + int(train_size/2) * [0])

    # create indices 
    indices = np.arange(X_sequences.shape[0])
    np.random.shuffle(indices)
    X_sequences = X_sequences[indices]
    y = y[indices]
    print("Sequences created!") 
    return X_sequences, test_sequences, y, nb_word, word_index


X_sequences, test_sequences, y, nb_word, word_index = create_sequences(max_, X, test)

65015
Sequences created!


In [ ]:
def create_glove_matrix(word_embedding, X, nb_word):
    '''
    This method creates an embedding matrix from a dict passed in argument
    '''
    X_splitted = [x.split() for x in X]

    glove_matrix = np.zeros((nb_word + 1, 200))
    for word, i in word_index.items():
        embedding_vector = word_embedding.get(word.encode())
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            glove_matrix[i] = embedding_vector
    print('Matrix created')
    
    return glove_matrix

In [ ]:
glove_matrix = create_glove_matrix(word_embedding, X,nb_word)

Matrix created


### Bidirectional LSTM(30), Activation RELU

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, Flatten
from tensorflow.python.keras.layers import LSTM, CuDNNLSTM
from keras.callbacks import EarlyStopping
import pyopencl as cl


callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)

model = Sequential()
model.add(Embedding(nb_word+1, 200, input_length=X_sequences.shape[1], weights=[glove_matrix]))
model.add(Bidirectional(LSTM(30)))
model.add(Flatten())
model.add(Dense(1, activation='relu'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 30, 200)           13003200  
_________________________________________________________________
bidirectional (Bidirectional (None, 60)                55440     
_________________________________________________________________
flatten (Flatten)            (None, 60)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 61        
Total params: 13,058,701
Trainable params: 13,058,701
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_sequences, y, validation_split=0.1, epochs=10, batch_size=128, verbose=1, shuffle=True,callbacks=[callback])

Epoch 1/10
1407/1407 [==============================] - 379s 269ms/step - loss: 0.5039 - accuracy: 0.7610 - val_loss: 0.4705 - val_accuracy: 0.7834
Epoch 2/10
1407/1407 [==============================] - 376s 267ms/step - loss: 0.4221 - accuracy: 0.8138 - val_loss: 0.4404 - val_accuracy: 0.8030
Epoch 3/10
1407/1407 [==============================] - 373s 265ms/step - loss: 0.3804 - accuracy: 0.8382 - val_loss: 0.6921 - val_accuracy: 0.7200
Epoch 4/10
1407/1407 [==============================] - 379s 269ms/step - loss: 0.3814 - accuracy: 0.8379 - val_loss: 0.4710 - val_accuracy: 0.7839
Epoch 5/10
1407/1407 [==============================] - 360s 256ms/step - loss: 0.3633 - accuracy: 0.8508 - val_loss: 0.5000 - val_accuracy: 0.8055
Epoch 6/10
1407/1407 [==============================] - 363s 258ms/step - loss: 0.3286 - accuracy: 0.8619 - val_loss: 0.5681 - val_accuracy: 0.8063
Epoch 7/10
1407/1407 [==============================] - 379s 270ms/step - loss: 0.2984 - accuracy: 0.8757 - val_

### LSTM(30), Activation RELU

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, Flatten
from tensorflow.python.keras.layers import LSTM, CuDNNLSTM
from keras.callbacks import EarlyStopping
import pyopencl as cl


callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)

model = Sequential()
model.add(Embedding(nb_word+1, 200, input_length=X_sequences.shape[1], weights=[glove_matrix]))
model.add(LSTM(30))
model.add(Flatten())
model.add(Dense(1, activation='relu'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 30, 200)           13003200  
_________________________________________________________________
lstm_1 (LSTM)                (None, 30)                27720     
_________________________________________________________________
flatten_1 (Flatten)          (None, 30)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 31        
Total params: 13,030,951
Trainable params: 13,030,951
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_sequences, y, validation_split=0.1, epochs=4, batch_size=128, verbose=1, shuffle=True,callbacks=[callback])

Epoch 1/4
1407/1407 [==============================] - 332s 236ms/step - loss: 0.5153 - accuracy: 0.7412 - val_loss: 0.4571 - val_accuracy: 0.7921
Epoch 2/4
1407/1407 [==============================] - 313s 222ms/step - loss: 0.4479 - accuracy: 0.7900 - val_loss: 0.4476 - val_accuracy: 0.8019
Epoch 3/4
1407/1407 [==============================] - 357s 254ms/step - loss: 0.4087 - accuracy: 0.8199 - val_loss: 0.4685 - val_accuracy: 0.7854
Epoch 4/4
1407/1407 [==============================] - 354s 251ms/step - loss: 0.4137 - accuracy: 0.8141 - val_loss: 0.4665 - val_accuracy: 0.7950


### Bidirectional LSTM(30), Activation sigmoid

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, Flatten
from tensorflow.python.keras.layers import LSTM, CuDNNLSTM
from keras.callbacks import EarlyStopping
import pyopencl as cl


callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)

model = Sequential()
model.add(Embedding(nb_word+1, 200, input_length=X_sequences.shape[1], weights=[glove_matrix]))
model.add(Bidirectional(LSTM(30)))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 30, 200)           13003200  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 60)                55440     
_________________________________________________________________
flatten_2 (Flatten)          (None, 60)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 61        
Total params: 13,058,701
Trainable params: 13,058,701
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_sequences, y, validation_split=0.1, epochs=4, batch_size=128, verbose=1, shuffle=True,callbacks=[callback])

Epoch 1/4
1407/1407 [==============================] - 508s 361ms/step - loss: 0.4298 - accuracy: 0.7896 - val_loss: 0.3921 - val_accuracy: 0.8152
Epoch 2/4
1407/1407 [==============================] - 499s 354ms/step - loss: 0.3553 - accuracy: 0.8372 - val_loss: 0.3910 - val_accuracy: 0.8148
Epoch 3/4
1407/1407 [==============================] - 481s 342ms/step - loss: 0.3061 - accuracy: 0.8627 - val_loss: 0.4098 - val_accuracy: 0.8154
Epoch 4/4
1407/1407 [==============================] - 479s 340ms/step - loss: 0.2649 - accuracy: 0.8841 - val_loss: 0.4408 - val_accuracy: 0.8091


### LSTM(30), Activation sigmoid

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, Flatten
from tensorflow.python.keras.layers import LSTM, CuDNNLSTM
from keras.callbacks import EarlyStopping
import pyopencl as cl


callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)

model = Sequential()
model.add(Embedding(nb_word+1, 200, input_length=X_sequences.shape[1], weights=[glove_matrix]))
model.add(LSTM(30))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 30, 200)           13003200  
_________________________________________________________________
lstm_3 (LSTM)                (None, 30)                27720     
_________________________________________________________________
flatten_3 (Flatten)          (None, 30)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 31        
Total params: 13,030,951
Trainable params: 13,030,951
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_sequences, y, validation_split=0.1, epochs=4, batch_size=128, verbose=1, shuffle=True,callbacks=[callback])

Epoch 1/4
1407/1407 [==============================] - 340s 241ms/step - loss: 0.4311 - accuracy: 0.7896 - val_loss: 0.3946 - val_accuracy: 0.8121
Epoch 2/4
1407/1407 [==============================] - 336s 239ms/step - loss: 0.3539 - accuracy: 0.8367 - val_loss: 0.3928 - val_accuracy: 0.8163
Epoch 3/4
1407/1407 [==============================] - 365s 260ms/step - loss: 0.3060 - accuracy: 0.8628 - val_loss: 0.4156 - val_accuracy: 0.8110
Epoch 4/4
1407/1407 [==============================] - 363s 258ms/step - loss: 0.2672 - accuracy: 0.8820 - val_loss: 0.4399 - val_accuracy: 0.8081


### Glove + GRU(30) + Activation Sigmoid

In [ ]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

np.random.seed(12)

model = Sequential()
model.add(Embedding(nb_word+1, 200, input_length=X_sequences.shape[1], weights=[glove_matrix]))
model.add(GRU(100, dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X_sequences, y, validation_split=0.1, epochs=2, batch_size=128, verbose=1, shuffle=True)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 30, 200)           13003200  
_________________________________________________________________
gru (GRU)                    (None, 100)               90600     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 13,093,901
Trainable params: 13,093,901
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/2
1407/1407 [==============================] - 374s 266ms/step - loss: 0.4334 - accuracy: 0.7860 - val_loss: 0.3895 - val_accuracy: 0.8163
Epoch 2/2
1407/1407 [==============================] - 374s 266ms/step - loss: 0.3636 - accuracy: 0.8312 - val_loss: 0.3867 - val_accuracy: 0.8191


### Glove + bidirectional GRU(30) + Activation Sigmoid

In [ ]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

np.random.seed(12)

model = Sequential()
model.add(Embedding(nb_word+1, 200, input_length=X_sequences.shape[1], weights=[glove_matrix]))
model.add(Bidirectional(GRU(100, dropout=0.2)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X_sequences, y, validation_split=0.1, epochs=2, batch_size=128, verbose=1, shuffle=True)


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 30, 200)           13003200  
_________________________________________________________________
bidirectional_2 (Bidirection (None, 200)               181200    
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 201       
Total params: 13,184,601
Trainable params: 13,184,601
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/2
1407/1407 [==============================] - 435s 309ms/step - loss: 0.4345 - accuracy: 0.7871 - val_loss: 0.3901 - val_accuracy: 0.8146
Epoch 2/2
1407/1407 [==============================] - 491s 349ms/step - loss: 0.3643 - accuracy: 0.8310 - val_loss: 0.3859 - val_accuracy: 0.8210


### Glove + GRU(30) + Activation Relu

In [ ]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

np.random.seed(12)

model = Sequential()
model.add(Embedding(nb_word+1, 200, input_length=X_sequences.shape[1], weights=[glove_matrix]))
model.add(GRU(100, dropout=0.2))
model.add(Dense(1, activation='relu'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 30, 200)           13003200  
_________________________________________________________________
gru_2 (GRU)                  (None, 100)               90600     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 101       
Total params: 13,093,901
Trainable params: 13,093,901
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.fit(X_sequences, y, validation_split=0.1, epochs=5, batch_size=128, verbose=1, shuffle=True)

Epoch 1/5
1407/1407 [==============================] - 434s 308ms/step - loss: 3.5997 - accuracy: 0.5967 - val_loss: 3.5394 - val_accuracy: 0.5789
Epoch 2/5
1407/1407 [==============================] - 373s 265ms/step - loss: 3.2143 - accuracy: 0.6135 - val_loss: 3.1619 - val_accuracy: 0.6077
Epoch 3/5
1407/1407 [==============================] - 365s 259ms/step - loss: 3.3820 - accuracy: 0.6213 - val_loss: 3.0970 - val_accuracy: 0.6205
Epoch 4/5
1407/1407 [==============================] - 383s 272ms/step - loss: 1.9264 - accuracy: 0.7131 - val_loss: 0.5561 - val_accuracy: 0.7773
Epoch 5/5
1407/1407 [==============================] - 382s 271ms/step - loss: 0.4639 - accuracy: 0.8147 - val_loss: 0.5456 - val_accuracy: 0.7882


### Glove + Bidirectional GRU(30) + Activation Relu

In [ ]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

np.random.seed(12)

model = Sequential()
model.add(Embedding(nb_word+1, 200, input_length=X_sequences.shape[1], weights=[glove_matrix]))
model.add(GRU(100, dropout=0.2))
model.add(Dense(1, activation='relu'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 30, 200)           13003200  
_________________________________________________________________
gru_2 (GRU)                  (None, 100)               90600     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 101       
Total params: 13,093,901
Trainable params: 13,093,901
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_sequences, y, validation_split=0.1, epochs=4, batch_size=128, verbose=1, shuffle=True,callbacks=[callback])

Epoch 1/4
1407/1407 [==============================] - 178s 126ms/step - loss: 0.7921 - accuracy: 0.7138 - val_loss: 0.5223 - val_accuracy: 0.7768
Epoch 2/4
1407/1407 [==============================] - 176s 125ms/step - loss: 0.4560 - accuracy: 0.7974 - val_loss: 0.4439 - val_accuracy: 0.8025
Epoch 3/4
1407/1407 [==============================] - 177s 126ms/step - loss: 0.6082 - accuracy: 0.7769 - val_loss: 0.4596 - val_accuracy: 0.7890
Epoch 4/4
1407/1407 [==============================] - 177s 126ms/step - loss: 0.4212 - accuracy: 0.8182 - val_loss: 0.4707 - val_accuracy: 0.7825


### Glove + Conv1D + MaxPool1D + Flatten

In [ ]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

model = Sequential()
model.add(Embedding(nb_word+1, 200, input_length=X_sequences.shape[1], weights=[glove_matrix]))
model.add(Convolution1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 30, 200)           13003200  
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 30, 32)            19232     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 15, 32)            0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 480)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 250)               120250    
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 251       
Total params: 13,142,933
Trainable params: 13,142,933
Non-trainable params: 0
__________________________________________

In [ ]:
model.fit(X_sequences, y, validation_split=0.1, epochs=4, batch_size=128, verbose=1, shuffle=True,callbacks=[callback])

Epoch 1/4
1407/1407 [==============================] - 177s 126ms/step - loss: 0.4340 - accuracy: 0.7852 - val_loss: 0.4017 - val_accuracy: 0.8099
Epoch 2/4
1407/1407 [==============================] - 173s 123ms/step - loss: 0.3510 - accuracy: 0.8384 - val_loss: 0.3976 - val_accuracy: 0.8154
Epoch 3/4
1407/1407 [==============================] - 175s 124ms/step - loss: 0.2899 - accuracy: 0.8712 - val_loss: 0.4306 - val_accuracy: 0.8087
Epoch 4/4
1407/1407 [==============================] - 178s 126ms/step - loss: 0.2319 - accuracy: 0.8987 - val_loss: 0.4799 - val_accuracy: 0.8025


### Glove + Conv1D + MaxPool1D + Bidirectional LSTM

In [ ]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

model = Sequential()
model.add(Embedding(nb_word+1, 50, input_length=X_sequences.shape[1]))
model.add(Convolution1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(LSTM(30)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 30, 50)            3250800   
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 30, 32)            4832      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 15, 32)            0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 60)                15120     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 61        
Total params: 3,270,813
Trainable params: 3,270,813
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_sequences, y, validation_split=0.1, epochs=4, batch_size=128, verbose=1, shuffle=True,callbacks=[callback])

Epoch 1/4
1407/1407 [==============================] - 51s 36ms/step - loss: 0.4334 - accuracy: 0.7900 - val_loss: 0.4021 - val_accuracy: 0.8082
Epoch 2/4
1407/1407 [==============================] - 49s 35ms/step - loss: 0.3468 - accuracy: 0.8419 - val_loss: 0.3961 - val_accuracy: 0.8158
Epoch 3/4
1407/1407 [==============================] - 48s 34ms/step - loss: 0.2757 - accuracy: 0.8798 - val_loss: 0.4252 - val_accuracy: 0.8095
Epoch 4/4
1407/1407 [==============================] - 48s 34ms/step - loss: 0.2156 - accuracy: 0.9082 - val_loss: 0.4824 - val_accuracy: 0.8093


### Glove + Conv1D + MaxPool1D + Bidirectional GRU

In [ ]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

model = Sequential()
model.add(Embedding(nb_word+1, 50, input_length=X_sequences.shape[1]))
model.add(Convolution1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(GRU(30)))
model.add(Dense
(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 30, 50)            3250800   
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 30, 32)            4832      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 15, 32)            0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 60)                11520     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 61        
Total params: 3,267,213
Trainable params: 3,267,213
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_sequences, y, validation_split=0.1, epochs=4, batch_size=128, verbose=1, shuffle=True,callbacks=[callback])

Epoch 1/4
1407/1407 [==============================] - 50s 35ms/step - loss: 0.4329 - accuracy: 0.7902 - val_loss: 0.3974 - val_accuracy: 0.8117
Epoch 2/4
1407/1407 [==============================] - 49s 35ms/step - loss: 0.3474 - accuracy: 0.8428 - val_loss: 0.3998 - val_accuracy: 0.8114
Epoch 3/4
1407/1407 [==============================] - 47s 34ms/step - loss: 0.2815 - accuracy: 0.8776 - val_loss: 0.4231 - val_accuracy: 0.8105
Epoch 4/4
1407/1407 [==============================] - 47s 34ms/step - loss: 0.2228 - accuracy: 0.9067 - val_loss: 0.4857 - val_accuracy: 0.8041
